In [1]:
#!pip install ffmpeg-python

In [1]:
import os
import gradio as gr
import tempfile
import whisper
from pydub import AudioSegment
from openai import OpenAI
from TTS.api import TTS

In [2]:
client = OpenAI(api_key="")

In [4]:
whisper_model = whisper.load_model("small")

In [5]:
tts_model = TTS("tts_models/multilingual/multi-dataset/xtts_v2")

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\TTS\tts\layers\xtts\xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.lo

In [6]:
LANGUAGES = ["French", "German", "Spanish", "Hindi", "Russian", "Chinese"]

LANGUAGE_CODES = {
    "french": "fr",
    "german": "de",
    "spanish": "es",
    "hindi": "hi",
    "russian": "ru",
    "chinese": "zh-cn"
}

os.makedirs("user_voices", exist_ok=True)
DEFAULT_VOICE_PATH = "user_voices/my_voice.wav"


def save_voice_sample(audio):
    if audio is None:
        return "⚠️ Please record a voice first.", None


    output_path = DEFAULT_VOICE_PATH
    AudioSegment.from_file(audio).export(output_path, format="wav")
    return f"✅ Voice sample saved as {output_path}.", output_path

def translate_from_english(audio, target_lang):
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        AudioSegment.from_file(audio).export(tmp.name, format="wav")
        audio_path = tmp.name

    result = whisper_model.transcribe(audio_path)
    english_text = result["text"]

    translation = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"Translate this from English to {target_lang}."},
            {"role": "user", "content": english_text}
        ]
    )

    translated_text = translation.choices[0].message.content
    lang_code = LANGUAGE_CODES.get(target_lang.lower(), target_lang.lower())

    speaker_wav = DEFAULT_VOICE_PATH if os.path.exists(DEFAULT_VOICE_PATH) else None

    audio_out_path = tempfile.NamedTemporaryFile(suffix=".wav", delete=False).name
    tts_model.tts_to_file(
        text=translated_text,
        speaker_wav=speaker_wav,
        language=lang_code,
        file_path=audio_out_path
    )

    return translated_text, audio_out_path


def translate_to_english(audio, source_lang):
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        AudioSegment.from_file(audio).export(tmp.name, format="wav")
        audio_path = tmp.name

    result = whisper_model.transcribe(audio_path)
    source_text = result["text"]

    translation = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"Translate this from {source_lang} to English."},
            {"role": "user", "content": source_text}
             
        ]
    )
    translated_text = translation.choices[0].message.content

    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as tmp_audio:
        english_audio_out = tmp_audio.name

    response = client.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="alloy",
        input=translated_text
    )

    response.stream_to_file(english_audio_out)

    return translated_text, english_audio_out

def text_translate(text, target_lang, direction):
    """Translate text in either direction (English <-> Target)."""
    
    if direction == "English → Target":
        src_lang = "English"
        tgt_lang = target_lang
    else:
        src_lang = target_lang
        tgt_lang = "English"

    translation = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    f"You are a professional translator. Translate ONLY the following text "
                    f"from {src_lang} to {tgt_lang}. "
                    f"Respond with ONLY the translated text — no explanations, no quotes, no extra words."
                )
            },
            {"role": "user", "content": text},
        ],
    )

    return translation.choices[0].message.content.strip()


with gr.Blocks(title="🌍 AI Voice Translator Studio") as demo:
    gr.Markdown("## 🎙️ Your AI Voice Translator — Speak, Record, and Communicate Effortlessly")
    
    with gr.Tab("🧠 1. Record & Save Your Voice"):
        gr.Markdown("Record your voice — the app will use it for translations from English.")
        voice_input = gr.Audio(sources=["microphone"], type="filepath", label="🎤 Record your voice sample")
        save_button = gr.Button("💾 Save Voice Sample")
        voice_status = gr.Textbox(label="Status", interactive=False)
        saved_voice_preview = gr.Audio(label="🔊 Your Saved Voice")
        save_button.click(save_voice_sample, inputs=voice_input, outputs=[voice_status, saved_voice_preview])

    with gr.Tab("🌍 2. English → Target Language (Your Voice)"):
        gr.Interface(
            fn=translate_from_english,
            inputs=[
                gr.Audio(sources=["microphone"], type="filepath", label="🎙️ Speak English"),
                gr.Dropdown(LANGUAGES, label="Translate to", value="French")
            ],
            outputs=[
                gr.Textbox(label="📝 Translated Text"),
                gr.Audio(label="🔊 Translation (Your Voice)", interactive=True, show_download_button=True)
            ]
        )

    with gr.Tab("🗣️ 3. Target Language → English (AI Voice)"):
        gr.Interface(
            fn=translate_to_english,
            inputs=[
                gr.Audio(sources=["microphone"], type="filepath", label="🎙️ Speak in Target Language"),
                gr.Dropdown(LANGUAGES, label="Source Language", value="French")
            ],
            outputs=[
                gr.Textbox(label="📝 English Translation"),
                gr.Audio(label="🔊 English Voice (GPT Voice)", interactive=True, show_download_button=True)
            ]
        )

    with gr.Tab("💬 4. Text Translator"):
        gr.Interface(
            fn=text_translate,
            inputs=[
                gr.Textbox(label="✍️ Enter text to translate"),
                gr.Dropdown(LANGUAGES, label="🌍 Choose Language", value="Spanish"),
                gr.Radio(["English → Target", "Target → English"], label="Translation Direction", value="English → Target")
            ],
            outputs=gr.Textbox(label="📝 Translated Text")
        )

demo.launch()
        

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 > Text splitted to sentences.
['Lib, −�� Douglas, − et rabroue sax sur le côté Corin mets tag gawon triomphe.']
 > Processing time: 59.74814987182617
 > Real-time factor: 5.801480944276083


Traceback (most recent call last):
  File "C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
  File "C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
  File "C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\gradio\blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\anyio\_backends\_asyncio.py"

 > Text splitted to sentences.
['Bonjour, je veux juste savoir si tu vas bien.']
 > Processing time: 46.48140907287598
 > Real-time factor: 6.627748771707936


C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 > Text splitted to sentences.
['Bonjour, je veux juste savoir comment tu vas.']
 > Processing time: 32.19431161880493
 > Real-time factor: 6.014339934887563


C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
C:\Users\USER\AppData\Local\Temp\ipykernel_12796\1620095288.py:84: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(english_audio_out)
C:\Users\USER\miniconda3\envs\audio_translate\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [5]:
#!pip uninstall -y numpy==2.2.6

In [6]:
#!python -c "import numpy; print(numpy.__version__)"


In [7]:
#!pip install numpy==1.22.0

In [8]:
#!python -c "import numpy; print(numpy.__version__)"


In [9]:
#pip install transformers==4.37.2 --force-reinstall

In [11]:
#!pip install sounddevice scipy

import sounddevice as sd
from scipy.io.wavfile import write

fs = 44100
seconds = 10

print("Recording... Speak now!")

recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1, dtype='int16')

sd.wait()

print("Recording complete!")

write("my_voice1.wav", fs, recording)
print("saved as my_voice1.wav")

from IPython.display import Audio

# use the filename parameter explicitly
Audio(filename="my_voice1.wav")


LANGUAGES = ["French", "German", "Spanish", "Hindi", "Russian", "Chinese"]

# Map display names → valid TTS language codes
LANGUAGE_CODES = {
    "french": "fr",
    "german": "de",
    "spanish": "es",
    "hindi": "hi",
    "russian": "ru",
    "chinese": "zh-cn"   # ✅ corrected
}

MY_VOICE_SAMPLE = "my_voice1.wav"

def translate_voice_note(audio, target_lang):
    import tempfile
    from pydub import AudioSegment

    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        AudioSegment.from_file(audio).export(tmp.name, format="wav")
        audio_path = tmp.name

    result = whisper_model.transcribe(audio_path)
    english_text = result["text"]

    # Translate text using GPT
    translation = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"Translate this from English to {target_lang}."},
            {"role": "user", "content": english_text}
        ]
    )

    translated_text = translation.choices[0].message.content

    # ✅ Correct the language code for TTS
    lang_code = LANGUAGE_CODES.get(target_lang.lower(), target_lang.lower())

    audio_out_path = tempfile.NamedTemporaryFile(suffix=".wav", delete=False).name
    tts_model.tts_to_file(
        text=translated_text,
        speaker_wav=MY_VOICE_SAMPLE,
        language=lang_code,
        file_path=audio_out_path  # ✅ was previously "audio" by mistake
    )

    return translated_text, audio_out_path


# Gradio interface
demo = gr.Interface(
    fn=translate_voice_note,
    inputs=[
        gr.Audio(sources=["microphone"], type="filepath", label="🎙️ Record your English voice note"),
        gr.Dropdown(LANGUAGES, label="🌍 Choose target language", value="Chinese")
    ],
    outputs=[
        gr.Textbox(label="📝 Translated Text"),
        gr.Audio(label="🔊 Translated Voice (Your Voice!)")
    ],
    title="🌍 Personal Voice Translator",
    description="Record an English voice note — it translates into your chosen language and speaks back in your own voice."
)

demo.launch()
